In [11]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


In [12]:
kdc_to_major = {
    "150": "심리학부", "200": "신학과", "300": "사회과학부", "320": "정치행정학부",
    "330": "경제경영학부", "340": "법학부", "350": "정치행정학부", "360": "사회과학부",
    "370": "교육학부", "410": "수학통계학부", "420": "물리학과", "427": "전자전기공학부",
    "430": "화학공학부", "440": "생명과학부", "450": "지구과학과", "510": "의학계열",
    "520": "간호보건학부", "530": "기계공학부", "540": "전자전기공학부", "550": "정보통신공학부",
    "560": "컴퓨터공학부", "570": "건축토목학부", "580": "화학공학부", "600": "융합공학과",
    "610": "농생명과학부", "620": "수의학과", "670": "식품영양학과", "700": "예술학부",
    "710": "회화과", "720": "디자인학부", "750": "음악학부", "770": "연극영화학부",
    "800": "언어학과", "810": "문예창작/국문학부", "820": "중문과", "830": "영어영문학부",
    "900": "역사문화학부"
}



In [13]:
# ✅ 2. 전처리 함수들
def extract_kdc_codes(text):
    if pd.isna(text):
        return []
    return re.findall(r"\d{1,3}(?:\.\d+)?", str(text))

def map_codes_to_major(code_list):
    for code in code_list:
        head = code.split('.')[0]
        if head in kdc_to_major:
            return kdc_to_major[head]
    return None



In [14]:
# ✅ 3. 데이터 불러오기
df = pd.read_csv("D:/workspace/project/data/filtered_books_utf8.csv", encoding="utf-8")
df["분류번호리스트"] = df["주제분류번호"].apply(extract_kdc_codes)
df["kdc_major"] = df["분류번호리스트"].apply(map_codes_to_major)
df = df.dropna(subset=["kdc_major", "도서명"])

# → 3자리 대표 분류번호 추출 (첫 번째 것만)
df["kdc3"] = df["분류번호리스트"].apply(lambda codes: codes[0].split('.')[0] if codes else None)



C:\Users\user\AppData\Local\Temp\ipykernel_28460\731957948.py:2: DtypeWarning: Columns (4,5,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("D:/workspace/project/data/filtered_books_utf8.csv", encoding="utf-8")


In [15]:
# ✅ 4. 학습/테스트 데이터 준비
X = df[["도서명", "kdc3"]]
y = df["kdc_major"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)



In [16]:
# ✅ 5. 전처리 파이프라인 구성
preprocessor = ColumnTransformer(transformers=[
    ("title_tfidf", TfidfVectorizer(), "도서명"),
    ("kdc_ohe", OneHotEncoder(handle_unknown="ignore"), ["kdc3"])
])



In [17]:
# ✅ 6. 모델 구성 (Logistic Regression + class_weight)
model = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("clf", LogisticRegression(max_iter=1000, class_weight="balanced"))
])



In [18]:
# ✅ 7. 학습
model.fit(X_train, y_train)


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('title_tfidf',
                                                  TfidfVectorizer(), '도서명'),
                                                 ('kdc_ohe',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['kdc3'])])),
                ('clf',
                 LogisticRegression(class_weight='balanced', max_iter=1000))])

In [19]:
# ✅ 8. 평가
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

      간호보건학부       1.00      1.00      1.00        63
      건축토목학부       1.00      1.00      1.00        27
      경제경영학부       1.00      1.00      1.00       211
        교육학부       1.00      1.00      1.00       437
       기계공학부       1.00      1.00      1.00       217
      농생명과학부       1.00      1.00      1.00       301
       디자인학부       1.00      1.00      1.00        62
   문예창작/국문학부       1.00      1.00      1.00       669
        물리학과       1.00      1.00      1.00       158
         법학부       1.00      1.00      1.00      1450
       사회과학부       1.00      1.00      1.00       305
       생명과학부       1.00      1.00      1.00        31
        수의학과       1.00      1.00      1.00        15
      수학통계학부       1.00      1.00      1.00       340
      식품영양학과       1.00      1.00      1.00       191
         신학과       1.00      1.00      1.00         8
        심리학부       1.00      1.00      1.00        39
        언어학과       1.00    

In [20]:
sample_titles = [
    "처음 읽는 전자기학 427",
    "열역학과 통계역학 420",
    "고전문학의 흐름 810",
    "행정학 개론 350",
    "디지털 회로 설계의 이해 550"
]

predicted = model.predict(pd.DataFrame({
    "도서명": [s.rsplit(" ", 1)[0] for s in sample_titles],
    "kdc3": [s.rsplit(" ", 1)[1].split('.')[0] for s in sample_titles]
}))

for s, p in zip(sample_titles, predicted):
    print(f"[{s}] → 예측 학과: {p}")


[처음 읽는 전자기학 427] → 예측 학과: 전자전기공학부
[열역학과 통계역학 420] → 예측 학과: 물리학과
[고전문학의 흐름 810] → 예측 학과: 문예창작/국문학부
[행정학 개론 350] → 예측 학과: 정치행정학부
[디지털 회로 설계의 이해 550] → 예측 학과: 정보통신공학부
